In [5]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [6]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [7]:
COLLECTION_NAME = 'data_true_2'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[4011928402,
 47152545,
 785949447813423104,
 15752235,
 874488674,
 49638970,
 22252902,
 1150738142678454274,
 2558218715,
 244459746,
 22151966,
 3230198245,
 16893929,
 5695632,
 117218629,
 1819261,
 17717552,
 93069110,
 139920793,
 58245058,
 22153494,
 186141547,
 546419860,
 1101685661591957504,
 1369341750,
 834428300260831232,
 2525358906,
 15526966,
 1086247732686262272,
 104797238,
 1267822628670042116,
 1291066586766487555,
 30740147,
 933069412705472512,
 3330129165,
 995999075030982656,
 585108974,
 1511018348,
 7768402,
 43078558,
 34101992,
 1658983280,
 2766603077,
 16012783,
 1282883336742952961,
 2460613650,
 954126001780518914,
 2193516251,
 984780184753651718,
 367040102,
 718563872806805504,
 958806212,
 1364579275387592712,
 756277679502917632,
 17217502,
 1389395601654108163,
 2998393649,
 1377761386340696064,
 167708017,
 1361436957251887114,
 16666454,
 30706941,
 3322601280,
 955525143157145600,
 15634751,
 5935682,
 1296645693071929344,
 10507282,
 4037654

In [8]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{874488674, 16893929, 196724330, 15752235, 966317909659103232, 16012783, 5695632, 403765458, 4011928402, 16351765, 16666454, 36876990}


NameError: name 'db' is not defined

In [9]:
users= [874488674, 16893929, 196724330, 15752235, 966317909659103232, 16012783, 5695632, 403765458, 4011928402, 16351765, 16666454, 36876990]

In [10]:
for user in users:
    print(user, get_followers_len(user, db))

874488674 9232
16893929 50064
196724330 6848
15752235 52110
966317909659103232 31102
16012783 60000
5695632 386907
403765458 0
4011928402 0
16351765 0
16666454 0
36876990 0


In [11]:
for user in users:
    print(user, get_cascade_size(user, 'data_true_2', db))

874488674 1
16893929 47
196724330 2
15752235 6
966317909659103232 25
16012783 22
5695632 712
403765458 1
4011928402 2
16351765 1
16666454 3
36876990 1


In [12]:
# Check if the text is same
db['data_true_2'].distinct('retweet_count')

[0, 1, 2, 3, 7, 25, 30, 54, 838]

In [13]:
db['data_true_2'].distinct('retweeted_status.retweet_count')

[1, 2, 3, 7, 25, 30, 54, 838]

In [14]:
db['data_true_2'].distinct('retweeted_status.quoted_status.retweet_count')

[189, 804]